In [ ]:
# 📦 Install required libraries
%pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
%pip install langchain-groq
%pip install sentence-transformers

In [ ]:
# 📥 Import necessary packages and modules
from langchain_openai import ChatOpenAI
import os
from crewai_tools import PDFSearchTool
from langchain_community.tools.tavily_search import TavilySearchResults
from crewai_tools  import tool
from crewai import Crew
from crewai import Task
from crewai import Agent

In [ ]:
# 🔐 Load API keys
GROQ_API_KEY='enter your groq api key'

In [ ]:
# 🧠 Set up the LLM model using Groq's Llama3
llm = ChatOpenAI(
    openai_api_base="https://api.groq.com/openai/v1",
    openai_api_key=GROQ_API_KEY,
    model_name="llama3-8b-8192",
    temperature=0.1,
    max_tokens=1000,
)

In [12]:
# 🔍 Define the PDF-based RAG tool with LLM and embedding model
rag_tool = PDFSearchTool(pdf='doc.pdf',
    config=dict(
        llm=dict(
            provider="groq", # or google, openai, anthropic, llama2, ...
            config=dict(
                model="llama3-8b-8192",
                # temperature=0.5,
                # top_p=1,
                # stream=true,
            ),
        ),
        embedder=dict(
            provider="huggingface", # or openai, ollama, ...
            config=dict(
                model="BAAI/bge-small-en-v1.5",
                #task_type="retrieval_document",
                # title="Embeddings",
            ),
        ),
    )
)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Inserting batches in chromadb: 100%|██████████| 1/1 [00:02<00:00,  2.45s/it]


In [13]:
# 🚀 Query the RAG tool to perform document-based search
rag_tool.run("What does Sporo Health do?")

Using Tool: Search a PDF's content


'Relevant Content:\nSporo Health Problem Doctors spend 2 hours per day on Manual Patient Chart Review (MCR), scanning through hundreds of patients’ documents and extracting critical puzzle pieces of information before the visit even starts. What We Do Sporo is a multimodal AI-driven solution that uses AI Agents to streamline MCR. What you get with investing at Sporo Health 1. Team: a. Vision: A highly motivated team inspired by the shortcomings of healthcare, dedicated towards the mission of bridging modern AI tech and clinical workflow. i. Refused offers from corporations like BlackRock and dropped-out of our full-time jobs to pursue our startup. ii. 5-year-vision: Building an AI-empowered EHR system for physicians - the AI clinical workflow that every PCP would know about. b. Resilience: i. Founders have never had an easy path in life. From a young age, we learned how to navigate the real world and understand what it takes to become self-made and shape our future. You will realize we

In [ ]:
TAVILY_API_KEY='enter the tavily key here'
# 🌐 Set up Tavily tool for real-time web search
web_search_tool = TavilySearchResults(k=3)

In [15]:
# 🔎 Run a web search using the Tavily tool
web_search_tool.run("What does Sporo Health do?")

[{'url': 'https://sporohealth.com/',
  'content': "Sporo's Synopsis. Searching through hundreds of patient chart documents and puzzle-piecing together a history is time-consuming and difficult - let Sporo do it for you within seconds, focusing on the key elements of a patient's story in an easily-digestible, familiar, presentation format."},
 {'url': 'https://sporohealth.com/about-us/',
  'content': 'At Sporo Health, he focuses on all things tech, ensuring continuous innovation and the responsible use of AI to revolutionize healthcare. Sonu is also the creative force behind the YouTube channel "AI Anytime," where he simplifies AI concepts for a global audience. Beyond his technological pursuits, he leads a dynamic community of over 40,000 ...'},
 {'url': 'https://www.linkedin.com/company/sporo-health',
  'content': "At Sporo Health, we're leveraging AI agents to revolutionize patient chart reviews. Our AI-driven solutions streamline administrative tasks, generate synopses from EHRs, an

In [16]:
@tool
def router_tool(question):
  """Router Function"""
  if 'Sporo Health' in question:
    return 'vectorstore'
  else:
    return 'web_search'

In [17]:
# 🤖 Define an AI agent with its role and toolset
Router_Agent = Agent(
  role='Router',
  goal='Route user question to a vectorstore or web search',
  backstory=(
    "You are an expert at routing a user question to a vectorstore or web search."
    "Use the vectorstore for questions on concept related to Retrieval-Augmented Generation."
    "You do not need to be stringent with the keywords in the question related to these topics. Otherwise, use web-search."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

In [18]:
# 🤖 Define an AI agent with its role and toolset
Retriever_Agent = Agent(
role="Retriever",
goal="Use the information retrieved from the vectorstore to answer the question",
backstory=(
    "You are an assistant for question-answering tasks."
    "Use the information present in the retrieved context to answer the question."
    "You have to provide a clear concise answer."
),
verbose=True,
allow_delegation=False,
llm=llm,
)

In [19]:
# 🤖 Define an AI agent with its role and toolset
Grader_agent =  Agent(
  role='Answer Grader',
  goal='Filter out erroneous retrievals',
  backstory=(
    "You are a grader assessing relevance of a retrieved document to a user question."
    "If the document contains keywords related to the user question, grade it as relevant."
    "It does not need to be a stringent test.You have to make sure that the answer is relevant to the question."
  ),
  verbose=True,
  allow_delegation=False,
  llm=llm,
)

In [20]:
# 🤖 Define an AI agent with its role and toolset
hallucination_grader = Agent(
    role="Hallucination Grader",
    goal="Filter out hallucination",
    backstory=(
        "You are a hallucination grader assessing whether an answer is grounded in / supported by a set of facts."
        "Make sure you meticulously review the answer and check if the response provided is in alignmnet with the question asked"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

In [21]:
# 🤖 Define an AI agent with its role and toolset
answer_grader = Agent(
    role="Answer Grader",
    goal="Filter out hallucination from the answer.",
    backstory=(
        "You are a grader assessing whether an answer is useful to resolve a question."
        "Make sure you meticulously review the answer and check if it makes sense for the question asked"
        "If the answer is relevant generate a clear and concise response."
        "If the answer gnerated is not relevant then perform a websearch using 'web_search_tool'"
    ),
    verbose=True,
    allow_delegation=False,
    llm=llm,
)

In [22]:
# 🧩 Define the task each agent is responsible for
router_task = Task(
    description=("Analyse the keywords in the question {question}"
    "Based on the keywords decide whether it is eligible for a vectorstore search or a web search."
    "Return a single word 'vectorstore' if it is eligible for vectorstore search."
    "Return a single word 'websearch' if it is eligible for web search."
    "Do not provide any other premable or explaination."
    ),
    expected_output=("Give a binary choice 'websearch' or 'vectorstore' based on the question"
    "Do not provide any other premable or explaination."),
    agent=Router_Agent,
    tools=[router_tool],
)

In [23]:
# 🧩 Define the task each agent is responsible for
retriever_task = Task(
    description=("Based on the response from the router task extract information for the question {question} with the help of the respective tool."
    "Use the web_serach_tool to retrieve information from the web in case the router task output is 'websearch'."
    "Use the rag_tool to retrieve information from the vectorstore in case the router task output is 'vectorstore'."
    ),
    expected_output=("You should analyse the output of the 'router_task'"
    "If the response is 'websearch' then use the web_search_tool to retrieve information from the web."
    "If the response is 'vectorstore' then use the rag_tool to retrieve information from the vectorstore."
    "Return a claer and consise text as response."),
    agent=Retriever_Agent,
    context=[router_task],
   #tools=[retriever_tool],
)

In [24]:
# 🧩 Define the task each agent is responsible for
grader_task = Task(
    description=("Based on the response from the retriever task for the quetion {question} evaluate whether the retrieved content is relevant to the question."
    ),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the document is relevant to the question"
    "You must answer 'yes' if the response from the 'retriever_task' is in alignment with the question asked."
    "You must answer 'no' if the response from the 'retriever_task' is not in alignment with the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=Grader_agent,
    context=[retriever_task],
)

In [25]:
# 🧩 Define the task each agent is responsible for
hallucination_task = Task(
    description=("Based on the response from the grader task for the quetion {question} evaluate whether the answer is grounded in / supported by a set of facts."),
    expected_output=("Binary score 'yes' or 'no' score to indicate whether the answer is sync with the question asked"
    "Respond 'yes' if the answer is in useful and contains fact about the question asked."
    "Respond 'no' if the answer is not useful and does not contains fact about the question asked."
    "Do not provide any preamble or explanations except for 'yes' or 'no'."),
    agent=hallucination_grader,
    context=[grader_task],
)

# 🧩 Define the task each agent is responsible for
answer_task = Task(
    description=("Based on the response from the hallucination task for the quetion {question} evaluate whether the answer is useful to resolve the question."
    "If the answer is 'yes' return a clear and concise answer."
    "If the answer is 'no' then perform a 'websearch' and return the response"),
    expected_output=("Return a clear and concise response if the response from 'hallucination_task' is 'yes'."
    "Perform a web search using 'web_search_tool' and return ta clear and concise response only if the response from 'hallucination_task' is 'no'."
    "Otherwise respond as 'Sorry! unable to find a valid response'."),
    context=[hallucination_task],
    agent=answer_grader,
    #tools=[answer_grader_tool],
)

In [26]:
# 👥 Assemble agents into a Crew to collaborate on the task
rag_crew = Crew(
    agents=[Router_Agent, Retriever_Agent, Grader_agent, hallucination_grader, answer_grader],
    tasks=[router_task, retriever_task, grader_task, hallucination_task, answer_task],
    verbose=True,

)

In [27]:
inputs ={"question":"Does Sporo Streamline patient chart reviews?"}

In [28]:
# ▶️ Start the collaborative process among the agents
result = rag_crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Router
 [INFO]: == Starting Task: Analyse the keywords in the question Does Sporo Streamline patient chart reviews?Based on the keywords decide whether it is eligible for a vectorstore search or a web search.Return a single word 'vectorstore' if it is eligible for vectorstore search.Return a single word 'websearch' if it is eligible for web search.Do not provide any other premable or explaination.


> Entering new CrewAgentExecutor chain...
Thought: The question seems to be related to a specific product or service, "Sporo", and its functionality in patient chart reviews.
Action: router_tool
Action Input: {"question": "Does Sporo Streamline patient chart reviews?"} 

web_search

Final Answer: websearch

> Finished chain.
 [DEBUG]: == [Router] Task output: websearch


 [DEBUG]: == Working Agent: Retriever
 [INFO]: == Starting Task: Based on the response from the router task extract information for the question Does Sporo Streamline patient chart reviews? with 

In [29]:
print(result)

Yes, Sporo Streamline patient chart reviews.
